This notebook helps find models in Biomodels that have hospitalization inside them - helping for giving ideas on how to write new compartments representing hospitalization into existing models (such as the one from Scenario 1).

What you need to do to get a useful hospitalization compartment:

- Make sure that infected population is stratified to symptomatic/asymptomatic
- Note that asymptomatic population will not get hospitalized
- Create new transitions from symptomatic infected to hospitalized. This runs in parallel to symptomatic infected going to recovered (no need to delete anything). Then, make a transition from hospitalized to recovered.
- If you're in a model that has death, then you also have to include another parallel edge to the dead node.

In [1]:
from mira.sources.biomodels import query_biomodels, get_template_model
from tqdm.auto import tqdm
from tabulate import tabulate
from textwrap import shorten
from mira.examples.sir import svir
from mira.modeling.viz import GraphicalModel

In [2]:
models_res = query_biomodels("submitter_keywords:COVID-19", limit=30)

In [ ]:
rows = []
for model_data in tqdm(models_res):
    try:
        template_model = get_template_model(model_data["id"])
    except Exception:
        pass
    rows.append((
        model_data["id"], model_data["name"], template_model,
    ))

  0%|          | 0/28 [00:00<?, ?it/s]

In [ ]:
len(rows)

In [ ]:
seen = set()
res_rows = []
for m_id, m_name, m in rows:
    for template in m.templates:
        for concept in template.get_concepts():
            if "hosp" in concept.name.lower():
                if m_id not in seen:
                    seen.add(m_id)
                    res_rows.append((
                        m_id, shorten(m_name, 40), len(m.templates),
                        concept.get_curie_str(), shorten(concept.name, 30)
                    ))
                    
print(tabulate(
    res_rows, 
    tablefmt="github", 
    headers=["model_id", "model_title", "model_size", "compartment_curie", "compartment_label"],
))

In [ ]:
m = next(x for i, _, x in rows if i == "BIOMD0000000960")

GraphicalModel.for_jupyter(m, width="30%")

In [ ]:
m = next(x for i, _, x in rows if i == "BIOMD0000000971")

GraphicalModel.for_jupyter(m, width="30%")

In [ ]:
m = next(x for i, _, x in rows if i == "BIOMD0000000981")

GraphicalModel.for_jupyter(m, width="50%")